In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [2]:
spark

In [3]:
df_pyspark = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [4]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

[Age,Experience] ---> new Feature ---> independent Feature

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age','Experience'],outputCol=['Independent Feature'])

In [8]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [9]:
output = featureassembler.transform(df_pyspark)
output.show()

+---------+---+----------+------+------------------------------------+
|     Name|age|Experience|Salary|VectorAssembler_eb6525b2bcf1__output|
+---------+---+----------+------+------------------------------------+
|    Krish| 31|        10| 30000|                         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|                          [30.0,8.0]|
|    Sunny| 29|         4| 20000|                          [29.0,4.0]|
|     Paul| 24|         3| 20000|                          [24.0,3.0]|
|   Harsha| 21|         1| 15000|                          [21.0,1.0]|
|  Shubham| 23|         2| 18000|                          [23.0,2.0]|
+---------+---+----------+------+------------------------------------+



In [12]:
# output = output.withColumnRenamed("VectorAssembler_eb6525b2bcf1__output",'Independent Feature')
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

In [13]:
final_data  = output.select('Independent Feature','salary')
final_data.show()

+-------------------+------+
|Independent Feature|salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression
## train_test_split
train_data,test_data = final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Feature',labelCol='salary')
regressor = regressor.fit(train_data)

In [18]:
regressor.coefficients

DenseVector([-90.5483, 1608.7819])

In [19]:
pred = regressor.evaluate(test_data)

In [24]:
pred.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|salary|       prediction|
+-------------------+------+-----------------+
|         [23.0,2.0]| 18000|17214.09079632846|
+-------------------+------+-----------------+



In [25]:
pred.meanAbsoluteError,pred.meanSquaredError,pred.rootMeanSquaredError

(785.909203671541, 617653.2764156357, 785.909203671541)